# Feature Engineering

In [1]:
import pandas as pd
import numpy as np

# 1. Load Data (Fast Reload)
orders = pd.read_csv('Data/olist_orders_dataset.csv')
items = pd.read_csv('Data/olist_order_items_dataset.csv')
sellers = pd.read_csv('Data/olist_sellers_dataset.csv')
customers = pd.read_csv('Data/olist_customers_dataset.csv')
geo = pd.read_csv('Data/olist_geolocation_dataset.csv')

# 2. Prepare Master Table (Python Version)
# Filter delivered and convert dates
orders = orders[orders['order_status'] == 'delivered'].copy()
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

# Create Target
orders['is_late'] = (orders['order_delivered_customer_date'] > orders['order_estimated_delivery_date']).astype(int)

# Merge
df = orders.merge(items, on='order_id')
df = df.merge(sellers, on='seller_id')
df = df.merge(customers, on='customer_id')

# 3. Feature Engineering: Geolocation & Distance
# Geo data has duplicates (many lat/lngs for one zip). We take the mean.
geo_agg = geo.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# Merge Seller Coords
df = df.merge(geo_agg, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'}, inplace=True)
df.drop(columns='geolocation_zip_code_prefix', inplace=True)

# Merge Customer Coords
df = df.merge(geo_agg, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'cust_lat', 'geolocation_lng': 'cust_lng'}, inplace=True)
df.drop(columns='geolocation_zip_code_prefix', inplace=True)

# Haversine Distance Function
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371  # Earth radius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return r * c

# Calculate Distance
df['distance_km'] = haversine_distance(df['seller_lat'], df['seller_lng'], df['cust_lat'], df['cust_lng'])

# 4. Feature Engineering: Seller Velocity (Behavioral Feature)
# "How often has this seller been late in the past?"
# We'll use a simple global average for this demo to avoid data leakage in a complex time-series split
seller_stats = df.groupby('seller_id')['is_late'].mean().reset_index()
seller_stats.rename(columns={'is_late': 'seller_late_probability'}, inplace=True)

# Merge back
df = df.merge(seller_stats, on='seller_id', how='left')

# Drop missing geo (approx 300 rows usually)
df_clean = df.dropna(subset=['distance_km', 'seller_late_probability'])

print("Feature Engineering Complete.")
print("New Features Added: ['distance_km', 'seller_late_probability']")
print(df_clean[['order_id', 'distance_km', 'seller_late_probability', 'is_late']].head())

Feature Engineering Complete.
New Features Added: ['distance_km', 'seller_late_probability']
                           order_id  distance_km  seller_late_probability  \
0  e481f51cbdc54678b7cc49136f2d6af7    18.576110                 0.000000   
1  53cdb2fc8bc7dce0b6741e2150273451   851.495069                 0.016000   
2  47770eb9100c2d0c44946d9cf07ec65d   514.410666                 0.115854   
3  949d5b44dbf5de918fe9c16f97b45f8a  1822.226336                 0.070968   
4  ad21c59c0840e6cb83a9ceb5573f8159    29.676625                 0.169591   

   is_late  
0        0  
1        0  
2        0  
3        0  
4        0  


## What I did?

1) distance_km (Route Risk): I merged the geolocation dataset (lat/long) for both Sellers and Customers and calculated the Haversine Distance.

Why? A seller in SP shipping to SP (10km) is very different from SP to Amazonia (3000km).

2) seller_late_probability (Seller Reliability): I calculated the historical percentage of late deliveries for each seller.

Why? Past behavior is the best predictor of future behavior. If a seller was late 50% of the time last month, they are a high risk today.